In [1]:
import numpy as np
np.set_printoptions(linewidth=100)
from joblib import Parallel, delayed
import sympy as sm
import sympy.abc as sbl
from scipy.sparse import diags
from scipy.integrate import RK45, solve_ivp
import scipy.sparse.linalg as la
import scipy.sparse as sp
import time
from functools import lru_cache
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm_notebook
from ipywidgets import HBox, IntSlider

plt.ioff()
plt.clf()

In [2]:
class Matrices:
    
    #A *alpha - beta(D *) +beta(E(1 - 0)alpha) = 0
    
    def __init__(self, N=10):
        
        self.N = N
        self.shape = (N+1, N+1)
        self.h = 1/N
        self.domain = np.linspace(0, 1, N+1) 
        self.beta = -1
        self.time_step = self.h**2
        self.runs = self.N**2
        self.time_point = 0
        
        self.A = (self.h) * sp.csc_matrix(diags([       [1/6 for i in range(N)],   \
                                        [1/3] + [2/3]*(N-1) + [1/3], \
                                             [1/6 for i in range(N)] ], \
                                                   [1, 0, -1]), dtype=np.float32)
        #-
        self.D = (1/self.h) * sp.csr_matrix(diags([      [-1 for i in range(N)],  \
                                      [1] + [2]*(N-1) + [1], \
                                        [-1 for i in range(N)]],  \
                             [1, 0, -1] ), dtype=np.float32)
                #+                
        row = np.array([0, 0, N, N])
        col = np.array([0, 1, N-1, N])
        data = np.array([1, -1, -1, 1 ])
        self.E = (1/self.h) *sp.csr_matrix((data, (row, col)), shape=self.shape, dtype=np.float32)    
      

        self.f_without_t = np.array(\
                                                               
                 [ (-(math.pi*self.h - math.sin(math.pi*self.h))*(math.pi**2*self.beta - 1)/\
                    (math.pi**2*self.h))] +\
                                     
                 [2*(math.pi**2*self.beta - 1)*(math.cos(math.pi*self.h) - 1)* \
                   math.sin(math.pi*self.h*k)/(math.pi**2*self.h)  for k in range(1, self.N)]+\
                                    
                [((math.pi**2*self.beta - 1)*(math.pi*self.h*math.cos(math.pi*self.h*self.N) \
                - math.sin(math.pi*self.h*self.N) + math.sin(math.pi*self.h*(self.N - 1)))/(math.pi**2*self.h)) ]
                   ).reshape(self.N+1, 1)
        
        self.f_t = lambda t:math.exp(t)*self.f_without_t
     

   
        #Set initial conditions here
        self.alpha = np.sin(np.pi* self.domain).reshape(N+1, 1)
        
        #Set solution here
        self.exact_solution = lambda t: np.sin(np.pi*self.domain).reshape(-1, 1).dot(np.exp(t).reshape(1, -1))
        

    

    
    def matB(self,i):
            if i == 0 :
                row = np.array([0, 0, 1, 1])
                col = np.array([0, 1, 0, 1])
                data = np.array([-1/3, 1/3, -1/6, 1/6])
                return sp.csr_matrix((data, (row, col)), shape=self.shape,dtype=np.float32)        
            elif i == self.N:
                row = np.array([self.N-1, self.N - 1, self.N, self.N])
                col = np.array([self.N-1, self.N , self.N -1, self.N])
                data = np.array([-1/6, 1/6, -1/3, 1/3])
                return sp.csr_matrix((data, (row, col)), shape=self.shape, dtype=np.float32)   
            else:
                row = np.array([i-1]*3 + [i]*3 + [i+1]*3)
                col = np.array([col for col in range(i-1, i+1 + 1)]*3)
                data = np.array([-1/6, 1/6, 0, -1/3, 0, 1/3, 0, -1/6, 1/6 ])
                return sp.csr_matrix((data, (row, col)), shape=self.shape, dtype=np.float32)    
            
   
    

In [3]:

class Methods(Matrices): 
    
    def __init__(self, num_basis = 10, method="Adam"):
        
        super().__init__(num_basis)
        eval("self."+method+"()")
        self.approx_half, self.approx_one = np.split(self.algo_y.flatten("F"), len(self.algo_t))  

        self.exact_half, self.exact_one =\
                            np.split(self.exact_solution(self.algo_t).flatten("F"), len(self.algo_t))

        self.error_half = math.sqrt(sum((self.approx_half - self.exact_half)**2))
        self.error_one = math.sqrt(sum((self.approx_one - self.exact_one)**2))
        
    

        
    def fun(self, t, y):
        f_t = self.f_t(t)
        y[0] = 0
        y[-1] = 0

        return la.inv(self.A).dot(self.beta*(self.D - self.E).dot(y) + f_t)
    
    def Adam(self):
        
        self.time_point = 0
        self.algo = solve_ivp(self.fun,t_span=(0, 1),  y0=self.alpha.ravel(), \
                     t_eval= [0.5, 1], vectorized=True, first_step = self.h, method="LSODA")    
        self.algo.y[[0, -1], :] = 0
        self.algo_y = self.algo.y
        self.algo_t = self.algo.t
        

    def RK4(self):
          
        self.time_point = 0
        self.algo_y = []
        self.algo_t = np.array([.5, 1])
        while(self.time_point != 1):
            k1 = self.fun( self.time_point, self.alpha)
            k2 = self.fun( self.time_point + self.time_step/2, self.alpha + k1/2 )
            k3 = self.fun( self.time_point + self.time_step/2, self.alpha + k2/2 )
            k4 = self.fun( self.time_point + self.time_step, self.alpha + k3 )
            print(k1, k2, k3 , k4)
            self.alpha = self.alpha + (1/6)*self.time_step*(k1 + 2*k2 + 2*k3 + k4)
            
            self.time_point += self.time_step
            if self.time_point in self.algo_t:
                self.algo_y.extend(self.alpha.ravel().tolist())
        self.algo_y = np.array(self.algo_y)
    
    def graph_half(self):
        return plt.plot(self.domain, approx_half, self.domain, exact_half)

    def graph_one(self):
        return plt.plot(self.domain, approx_one, self.domain, exact_one)



In [ ]:
heateqn = Methods(10, method="RK4")

[[ 1.78194539]
 [-0.16591581]
 [ 0.72062034]
 [ 0.78123501]
 [ 0.9687491 ]
 [ 1.00332901]
 [ 0.96874903]
 [ 0.78123509]
 [ 0.72062028]
 [-0.16591578]
 [ 1.78194536]] [[ -57.89332249]
 [ 119.20165237]
 [-101.23861275]
 [  41.66474988]
 [ -22.27395744]
 [   7.51874055]
 [ -22.27393457]
 [  41.66472201]
 [-101.23858335]
 [ 119.20162148]
 [ -57.89331211]] [[ 21081.85320737]
 [-42160.29140757]
 [ 45668.67761126]
 [-31507.65234965]
 [ 18314.3929125 ]
 [-13624.52795473]
 [ 18314.38501088]
 [-31507.64251473]
 [ 45668.66330395]
 [-42160.27522133]
 [ 21081.84679374]] [[-17209132.10631078]
 [ 34418267.64493883]
 [-42470380.85898603]
 [ 36460090.00176529]
 [-27170954.15011585]
 [ 23167151.26718787]
 [-27170948.78822727]
 [ 36460083.35834686]
 [-42470368.2446399 ]
 [ 34418252.54392313]
 [-17209125.86783285]]
[[ 25659324.48835148]
 [-51318645.54487948]
 [ 68567644.48006205]
 [-67462109.82688147]
 [ 58972877.36686804]
 [-54623575.24878504]
 [ 58972871.9269236 ]
 [-67462103.77371082]
 [ 68567627.44185

 [-2.38887065e+86]]
[[ 4.48747312e+86]
 [-8.97494625e+86]
 [ 1.42834794e+87]
 [-1.84205136e+87]
 [ 2.10467371e+87]
 [-2.19467511e+87]
 [ 2.10467416e+87]
 [-1.84205217e+87]
 [ 1.42834856e+87]
 [-8.97494882e+86]
 [ 4.48747472e+86]] [[-2.52889696e+89]
 [ 5.05779392e+89]
 [-8.04939593e+89]
 [ 1.03808046e+90]
 [-1.18608020e+90]
 [ 1.23680013e+90]
 [-1.18608046e+90]
 [ 1.03808091e+90]
 [-8.04939941e+89]
 [ 5.05779537e+89]
 [-2.52889786e+89]] [[ 1.42768284e+92]
 [-2.85536567e+92]
 [ 4.54426757e+92]
 [-5.86045886e+92]
 [ 6.69598800e+92]
 [-6.98232615e+92]
 [ 6.69598944e+92]
 [-5.86046145e+92]
 [ 4.54426954e+92]
 [-2.85536649e+92]
 [ 1.42768334e+92]] [[-1.61198492e+95]
 [ 3.22396983e+95]
 [-5.13089505e+95]
 [ 6.61699580e+95]
 [-7.56038488e+95]
 [ 7.88368693e+95]
 [-7.56038650e+95]
 [ 6.61699872e+95]
 [-5.13089727e+95]
 [ 3.22397076e+95]
 [-1.61198549e+95]]
[[ 3.02810103e+95]
 [-6.05620207e+95]
 [ 9.63834616e+95]
 [-1.24299748e+96]
 [ 1.42021238e+96]
 [-1.48094442e+96]
 [ 1.42021269e+96]
 [-1.24

 [-7.33580721e+168]] [[ 4.14141129e+171]
 [-8.28282258e+171]
 [ 1.31819762e+172]
 [-1.69999738e+172]
 [ 1.94236704e+172]
 [-2.02542779e+172]
 [ 1.94236746e+172]
 [-1.69999813e+172]
 [ 1.31819819e+172]
 [-8.28282496e+171]
 [ 4.14141276e+171]] [[-4.67603335e+174]
 [ 9.35206671e+174]
 [-1.48836607e+175]
 [ 1.91945303e+175]
 [-2.19311062e+175]
 [ 2.28689383e+175]
 [-2.19311109e+175]
 [ 1.91945388e+175]
 [-1.48836671e+175]
 [ 9.35206940e+174]
 [-4.67603502e+174]]
[[ 8.78389205e+174]
 [-1.75677841e+175]
 [ 2.79588400e+175]
 [-3.60567749e+175]
 [ 4.11974112e+175]
 [-4.29591216e+175]
 [ 4.11974201e+175]
 [-3.60567909e+175]
 [ 2.79588521e+175]
 [-1.75677892e+175]
 [ 8.78389518e+174]] [[-4.95012618e+177]
 [ 9.90025236e+177]
 [-1.57560891e+178]
 [ 2.03196470e+178]
 [-2.32166314e+178]
 [ 2.42094360e+178]
 [-2.32166364e+178]
 [ 2.03196560e+178]
 [-1.57560960e+178]
 [ 9.90025521e+177]
 [-4.95012794e+177]] [[ 2.79458213e+180]
 [-5.58916426e+180]
 [ 8.89506318e+180]
 [-1.14714091e+181]
 [ 1.31068948e+

 [ 1.20133782e+251]] [[-1.35642015e+254]
 [ 2.71284029e+254]
 [-4.31744080e+254]
 [ 5.56793454e+254]
 [-6.36175835e+254]
 [ 6.63380397e+254]
 [-6.36175971e+254]
 [ 5.56793700e+254]
 [-4.31744267e+254]
 [ 2.71284107e+254]
 [-1.35642063e+254]]
[[ 2.54802463e+254]
 [-5.09604925e+254]
 [ 8.11027874e+254]
 [-1.04593214e+255]
 [ 1.19505132e+255]
 [-1.24615488e+255]
 [ 1.19505158e+255]
 [-1.04593261e+255]
 [ 8.11028225e+254]
 [-5.09605072e+254]
 [ 2.54802553e+254]] [[-1.43592878e+257]
 [ 2.87185756e+257]
 [-4.57051417e+257]
 [ 5.89430751e+257]
 [-6.73466252e+257]
 [ 7.02265451e+257]
 [-6.73466396e+257]
 [ 5.89431012e+257]
 [-4.57051615e+257]
 [ 2.87185839e+257]
 [-1.43592929e+257]] [[ 8.10650227e+259]
 [-1.62130045e+260]
 [ 2.58027306e+260]
 [-3.32761749e+260]
 [ 3.80203793e+260]
 [-3.96462314e+260]
 [ 3.80203875e+260]
 [-3.32761896e+260]
 [ 2.58027418e+260]
 [-1.62130092e+260]
 [ 8.10650516e+259]] [[-9.15298490e+262]
 [ 1.83059698e+263]
 [-2.91336505e+263]
 [ 3.75718548e+263]
 [-4.29284969e+

c:\users\dell\desktop\python~1\maths\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in add


...

 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]


 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]


 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]


 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]


 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]] [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]


In [ ]:
heateqn.approx_half


errors = []
for i in tqdm_notebook([2**j for j in range(2, 8)]):
    heateqn = Methods(i, method="RK4") 
    errors.append([
          heateqn.error_half,
         heateqn.error_one
    ])
    print("Done")
    
    